## 1. Procesamiento de Auto y Transporte de Carga (DD + DI)

In [1]:
import pandas as pd
import numpy as np
import re
import gc
from pathlib import Path

# ==============================================================================
# 1. CONFIGURACIÓN - AUTO Y CARGA
# ==============================================================================
print("🚗 INICIANDO PROCESAMIENTO: AUTO Y CARGA...")

VISUM_BASE_DIR = Path(r"D:\NB3_VISUM plus MOVES merge\NB3_OutputData\VISUM Files V2\Link files with County\20251030\Auto\Links")
INTEGRATION_DIR = Path(r"D:\NB3_VISUM plus MOVES merge\Integration Files V1")
INTEGRATION_DIR.mkdir(parents=True, exist_ok=True)

RUTA_FACTORES_DD = INTEGRATION_DIR / "Output Files" / "Generated emission rates" / "Lookup_Table_Emission_Rates_g_per_km.parquet"
RUTA_FACTORES_DI = INTEGRATION_DIR / "Output Files" / "Generated emission rates" / "Lookup_Table_Emission_Rates_g_per_veh.parquet"

FILES_MAPPING = {
    "EM": "scenario=AutoEM", "AP": "scenario=AutoAP", "LM": "scenario=AutoLM",
    "MP": "scenario=AutoMP", "EA": "scenario=AutoEA", "PP": "scenario=AutoPP", "EV": "scenario=AutoEV"
}

HOURS_MAPPING = {
    "EM": [22, 23, 0, 1, 2, 3, 4, 5], "AP": [6, 7, 8, 9], "LM": [10, 11],
    "MP": [12], "EA": [13, 14, 15, 16], "PP": [17, 18], "EV": [19, 20, 21]
}

VOLUME_FACTORS = {
    "EM": 1.0, "AP": 0.290781, "LM": 0.503228, "MP": 1.0, 
    "EA": 0.250912, "PP": 0.512564, "EV": 0.406226 
}

LDVS_HDVS_A_MAPPING = {
    11: 0.146429251, 21: 0.320888794, 22: 0.104594992,
    31: 0.190505261, 32: 0.220278756, 43: 0.002389634, 44: 0.014913313
}

POLLUTANTS = ['CO2', 'CO2_Equiv', 'CO', 'NOx', 'Total_PM25', 'Total_PM10', 'TotalHC', 'CH4', 'N2O']

def clean_visum_string(val):
    if pd.isna(val) or val == "": return 0.0
    if isinstance(val, (int, float)): return float(val)
    return float(re.sub(r'[^\d\.]', '', str(val).lower()))

def get_moves_speed_bin(speed_kmh):
    speed_mph = speed_kmh / 1.60934
    bins = [0, 2.5, 7.5, 12.5, 17.5, 22.5, 27.5, 32.5, 37.5, 42.5, 47.5, 52.5, 57.5, 62.5, 67.5, 72.5]
    for i, limite in enumerate(bins[1:], start=1):
        if speed_mph < limite: return i
    return 16

# ==============================================================================
# 2. PROCESAMIENTO DD (DISTANCE DEPENDENT)
# ==============================================================================
print("\n--- A. Calculando Emisiones Dependientes de la Distancia (DD) ---")
df_ef_dd = pd.read_parquet(RUTA_FACTORES_DD)

# 🌟 MAGIA DE NOMBRES: Renombramos las columnas para que empaten perfectamente
df_ef_dd = df_ef_dd.rename(columns={'PM10': 'Total_PM10', 'PM25': 'Total_PM25', 'HC': 'TotalHC'})

cols_presentes = [c for c in POLLUTANTS if c in df_ef_dd.columns]
ef_lookup_dd = df_ef_dd.groupby(['Source', 'SpeedBin'])[cols_presentes].mean()

all_hourly_emissions_dd = []
geometry_records = []
procesed_links = set()
all_visum_dfs = [] 

for periodo, folder_name in FILES_MAPPING.items():
    found_files = list((VISUM_BASE_DIR / folder_name).glob("Links.parquet"))
    if not found_files: continue
        
    df_links = pd.read_parquet(found_files[0])
    if 'NO' in df_links.columns: df_links.rename(columns={'NO': 'LinkID'}, inplace=True)
    elif 'no' in df_links.columns: df_links.rename(columns={'no': 'LinkID'}, inplace=True)
    
    df_links['L_km'] = df_links['LENGTH'].apply(clean_visum_string)
    col_spd = next((c for c in df_links.columns if 'VCUR' in c.upper() and 'AUTO_C' in c.upper()), None)
    df_links['V_kmh'] = df_links[col_spd].apply(clean_visum_string) if col_spd else 0.0
    df_links['SpeedBin'] = df_links['V_kmh'].apply(get_moves_speed_bin)
    
    col_vol = next((c for c in df_links.columns if 'VOLVEH' in c.upper() and 'AUTO_C' in c.upper()), None)
    df_links['Q_periodo'] = df_links[col_vol].apply(clean_visum_string) if col_vol else 0.0
    
    factor_vol = VOLUME_FACTORS.get(periodo, 1.0)
    df_links['VOL_AUTO_C'] = (df_links['Q_periodo'] / factor_vol) if factor_vol not in [0, 1.0] else df_links['Q_periodo']
    df_links['VOL_CA1_C'] = df_links[next((c for c in df_links.columns if 'VOL' in c.upper() and 'CA1_C' in c.upper()), 'Q_periodo')].apply(clean_visum_string) / factor_vol
    df_links['VOL_CA2_C'] = df_links[next((c for c in df_links.columns if 'VOL' in c.upper() and 'CA2_C' in c.upper()), 'Q_periodo')].apply(clean_visum_string) / factor_vol
    df_links['VOL_CA_BD'] = df_links[next((c for c in df_links.columns if 'VOL' in c.upper() and 'CA_BD' in c.upper()), 'Q_periodo')].apply(clean_visum_string) / factor_vol
    df_links['VOL_CU_C'] = df_links[next((c for c in df_links.columns if 'VOL' in c.upper() and 'CU_C' in c.upper()), 'Q_periodo')].apply(clean_visum_string) / factor_vol
    df_links['Periodo'] = periodo
    all_visum_dfs.append(df_links[['LinkID', 'Periodo', 'VOL_AUTO_C', 'VOL_CA1_C', 'VOL_CA2_C', 'VOL_CA_BD', 'VOL_CU_C']].copy())

    if 'WKTPOLYWGS84' in df_links.columns:
        nuevos_links = df_links[~df_links['LinkID'].isin(procesed_links)]
        if not nuevos_links.empty:
            geometry_records.append(nuevos_links[['LinkID', 'WKTPOLYWGS84']].copy())
            procesed_links.update(nuevos_links['LinkID'].tolist())

    horas_periodo = HOURS_MAPPING.get(periodo, [])
    if len(horas_periodo) == 0: continue

    df_activos = df_links[df_links['Q_periodo'] > 0].copy()
    for source_id, fraction in LDVS_HDVS_A_MAPPING.items():
        vol_hora_source = (df_activos['Q_periodo'] * fraction) / len(horas_periodo)
        try: factor_df = ef_lookup_dd.loc[source_id]
        except KeyError: continue 
            
        temp_emissions = pd.DataFrame({'LinkID': df_activos['LinkID']})
        for pol in cols_presentes:
            temp_emissions[pol] = vol_hora_source * df_activos['L_km'] * df_activos['SpeedBin'].map(factor_df[pol]).fillna(0)
            
        for h in horas_periodo:
            df_h = temp_emissions.copy()
            df_h['Hour'] = h + 1 
            df_h['Source'] = source_id
            df_h['Day'] = 5 
            all_hourly_emissions_dd.append(df_h)

df_final_dd = pd.concat(all_hourly_emissions_dd, ignore_index=True)
df_final_dd.to_parquet(INTEGRATION_DIR / "Link_Emissions_DistBased_Day_5.parquet", index=False)
pd.concat(geometry_records, ignore_index=True).to_parquet(INTEGRATION_DIR / "Links_Geometry_Master.parquet", index=False)
print("   ✅ Emisiones DD y Geometría de Auto guardadas.")
del all_hourly_emissions_dd, df_final_dd; gc.collect()

# ==============================================================================
# 3. PROCESAMIENTO DI OPTIMIZADO (VECTORIZADO)
# ==============================================================================
print("\n--- B. Calculando Emisiones Independientes de la Distancia (DI) ---")
df_visum_master = pd.concat(all_visum_dfs, ignore_index=True)
del all_visum_dfs; gc.collect()

df_long = df_visum_master.melt(id_vars=['LinkID', 'Periodo'],
                               value_vars=['VOL_AUTO_C', 'VOL_CA1_C', 'VOL_CA2_C', 'VOL_CA_BD', 'VOL_CU_C'],
                               var_name='TSYS', value_name='Volume')
df_long = df_long[df_long['Volume'] > 0]
del df_visum_master; gc.collect()

map_rows = []
total_auto = sum(LDVS_HDVS_A_MAPPING.values())
for k, v in LDVS_HDVS_A_MAPPING.items():
    map_rows.append({'TSYS': 'VOL_AUTO_C', 'Source': k, 'Share': v / total_auto})
map_rows.extend([
    {'TSYS': 'VOL_CA1_C', 'Source': 61, 'Share': 1.0},
    {'TSYS': 'VOL_CA2_C', 'Source': 61, 'Share': 1.0},
    {'TSYS': 'VOL_CA_BD', 'Source': 61, 'Share': 1.0},
    {'TSYS': 'VOL_CU_C', 'Source': 52, 'Share': 1.0}
])
df_map = pd.DataFrame(map_rows)

df_long = df_long.merge(df_map, on='TSYS', how='inner')
df_long['Source_Volume'] = df_long['Volume'] * df_long['Share']
df_long = df_long.groupby(['LinkID', 'Periodo', 'Source'], as_index=False)['Source_Volume'].sum()

df_totals = df_long.groupby(['Periodo', 'Source'], as_index=False)['Source_Volume'].sum()
df_totals.rename(columns={'Source_Volume': 'Total_Volume'}, inplace=True)
df_long = df_long.merge(df_totals, on=['Periodo', 'Source'])
df_long['Network_Fraction'] = (df_long['Source_Volume'] / df_long['Total_Volume']).astype('float32')
df_long.drop(columns=['Source_Volume', 'Total_Volume'], inplace=True)
gc.collect()

df_rates = pd.read_parquet(RUTA_FACTORES_DI)
df_day = df_rates[df_rates['Day'] == 5].copy()
rate_cols = [c for c in df_rates.columns if c.startswith('Rate_')]
macro_cols = ['Hour', 'Source']
for col in rate_cols:
    base = col.replace('Rate_', '').replace('_per_veh', '')
    df_day[base] = df_day[col] * df_day['sourceTypePopulation']
    macro_cols.append(base)
emission_cols = [c for c in macro_cols if c not in ['Hour', 'Source']]

final_di_chunks = []
for periodo, horas in HOURS_MAPPING.items():
    if not horas: continue
    df_periodo = df_long[df_long['Periodo'] == periodo]
    if df_periodo.empty: continue

    for h in horas:
        hour_val = h + 1
        df_macro_h = df_day[df_day['Hour'] == hour_val]
        if df_macro_h.empty: continue

        df_merged = pd.merge(df_periodo, df_macro_h[macro_cols], on='Source', how='inner')
        for col in emission_cols:
            df_merged[col] = df_merged[col] * df_merged['Network_Fraction']

        df_merged['Hour'] = hour_val
        df_merged['Day'] = 5
        final_di_chunks.append(df_merged[['LinkID', 'Day', 'Hour', 'Source'] + emission_cols])

df_final_di = pd.concat(final_di_chunks, ignore_index=True)
df_final_di.to_parquet(INTEGRATION_DIR / "Link_Emissions_VolBased_Day_5.parquet", index=False)
print("   ✅ Emisiones DI de Auto guardadas.\n")
del df_long, final_di_chunks, df_final_di; gc.collect()

🚗 INICIANDO PROCESAMIENTO: AUTO Y CARGA...

--- A. Calculando Emisiones Dependientes de la Distancia (DD) ---
   ✅ Emisiones DD y Geometría de Auto guardadas.

--- B. Calculando Emisiones Independientes de la Distancia (DI) ---
   ✅ Emisiones DI de Auto guardadas.



0

## 2. Procesamiento de Transporte Público (DD + DI Continuo)

In [2]:
import pandas as pd
import numpy as np
import re
import gc
from pathlib import Path

# ==============================================================================
# 1. CONFIGURACIÓN - TRANSPORTE PÚBLICO
# ==============================================================================
print("🚌 INICIANDO PROCESAMIENTO: TRANSPORTE PÚBLICO...")

DIR_PROFILES = Path(r"D:\VISUM\Archivos de integración MOVES_VISUM\InputData\20251002\Transit\Timeprofiles")
DIR_TP_ITEMS = Path(r"D:\VISUM\Archivos de integración MOVES_VISUM\InputData\20251002\Transit\Timeprofileitems")
DIR_LR_ITEMS = Path(r"D:\VISUM\Archivos de integración MOVES_VISUM\InputData\20251002\Transit\Linerouteitems")

INTEGRATION_DIR = Path(r"D:\NB3_VISUM plus MOVES merge\Integration Files V1")
RUTA_FACTORES_DD = INTEGRATION_DIR / "Output Files" / "Generated emission rates" / "Lookup_Table_Emission_Rates_g_per_km.parquet"
RUTA_FACTORES_DI = INTEGRATION_DIR / "Output Files" / "Generated emission rates" / "Lookup_Table_Emission_Rates_g_per_veh.parquet"

TRANSIT_BUS_ID = 42
HEADWAY_IS_SECONDS = True 
POLLUTANTS = ['CO2', 'CO2_Equiv', 'CO', 'NOx', 'Total_PM25', 'Total_PM10', 'TotalHC', 'CH4', 'N2O']

FOLDER_MAPPING = {
    "EM": "scenario=TransitEM", "AP": "scenario=TransitAP", "LM": "scenario=TransitLM",
    "MP": "scenario=TransitMP", "EA": "scenario=TransitEA", "PP": "scenario=TransitPP", "EV": "scenario=TransitEV"
}
HOURS_MAPPING = {
    "EM": [22, 23, 0, 1, 2, 3, 4, 5], "AP": [6, 7, 8, 9], "LM": [10, 11],
    "MP": [12], "EA": [13, 14, 15, 16], "PP": [17, 18], "EV": [19, 20, 21]
}

def clean_visum_string(val):
    if pd.isna(val) or val == "": return 0.0
    if isinstance(val, (int, float)): return float(val)
    return float(re.sub(r'[^\d\.]', '', str(val).lower()))

def get_moves_speed_bin(speed_kmh):
    speed_mph = speed_kmh / 1.60934
    bins = [0, 2.5, 7.5, 12.5, 17.5, 22.5, 27.5, 32.5, 37.5, 42.5, 47.5, 52.5, 57.5, 62.5, 67.5, 72.5]
    for i, limite in enumerate(bins[1:], start=1):
        if speed_mph < limite: return i
    return 16

# ==============================================================================
# 2. PROCESAMIENTO DD (RUTAS CONTINUAS)
# ==============================================================================
print("\n--- A. Calculando Emisiones Dependientes de la Distancia (DD) TP ---")
df_ef = pd.read_parquet(RUTA_FACTORES_DD)

# 🌟 MAGIA DE NOMBRES PARA TP TAMBIÉN
df_ef = df_ef.rename(columns={'PM10': 'Total_PM10', 'PM25': 'Total_PM25', 'HC': 'TotalHC'})

cols_presentes = [c for c in POLLUTANTS if c in df_ef.columns]
ef_lookup = df_ef.groupby(['Source', 'SpeedBin'])[cols_presentes].mean()

all_hourly_emissions = []
wkt_to_id = {} 
current_link_id = 9000000 
geometry_records = []

for periodo, folder in FOLDER_MAPPING.items():
    file_prof = list((DIR_PROFILES / folder).glob("*.parquet"))
    file_tp = list((DIR_TP_ITEMS / folder).glob("*.parquet"))
    file_lr = list((DIR_LR_ITEMS / folder).glob("*.parquet"))
    
    if not (file_prof and file_tp and file_lr): continue
        
    df_prof = pd.read_parquet(file_prof[0])
    df_tp = pd.read_parquet(file_tp[0])
    df_lr = pd.read_parquet(file_lr[0])
    
    df_prof.columns = [c.strip().upper() for c in df_prof.columns]
    df_tp.columns = [c.strip().upper() for c in df_tp.columns]
    df_lr.columns = [c.strip().upper() for c in df_lr.columns]
    
    merge_keys = ['LINENAME', 'LINEROUTENAME', 'DIRECTIONCODE']
    df_merged = pd.merge(df_lr, df_prof, on=merge_keys, how='inner')
    df_merged = pd.merge(df_merged, df_tp[merge_keys + ['LRITEMINDEX', 'VEL']], 
                         left_on=merge_keys + ['INDEX'], right_on=merge_keys + ['LRITEMINDEX'], how='left')
    
    df_merged.sort_values(merge_keys + ['INDEX'], inplace=True)
    df_merged['VEL'] = df_merged.groupby(merge_keys)['VEL'].ffill().bfill()
    df_merged = df_merged[df_merged['WKTPOLYWGS84'].notna() & (df_merged['WKTPOLYWGS84'] != 'LINESTRING EMPTY')]
    
    horas = HOURS_MAPPING.get(periodo, [])
    hw_col = f"HEADWAY{periodo}"
    if len(horas) == 0 or hw_col not in df_merged.columns: continue
        
    df_merged['HEADWAY_SEC'] = df_merged[hw_col].apply(clean_visum_string) * (1.0 if HEADWAY_IS_SECONDS else 60.0)
    df_merged['Vol_Hora'] = np.where(df_merged['HEADWAY_SEC'] > 0, 3600 / df_merged['HEADWAY_SEC'], 0)
    df_merged['L_km'] = df_merged['POSTLENGTH'].apply(clean_visum_string)
    df_merged['V_kmh'] = df_merged['VEL'].apply(clean_visum_string)
    df_merged['SpeedBin'] = df_merged['V_kmh'].apply(get_moves_speed_bin)
    
    df_activos = df_merged[df_merged['Vol_Hora'] > 0].copy()
    if df_activos.empty: continue
        
    for idx, row in df_activos.iterrows():
        wkt = row['WKTPOLYWGS84']
        if wkt not in wkt_to_id:
            wkt_to_id[wkt] = current_link_id
            geometry_records.append({'LinkID': current_link_id, 'WKTPOLYWGS84': wkt})
            current_link_id += 1
        df_activos.at[idx, 'LinkID'] = wkt_to_id[wkt]

    try: factor_df = ef_lookup.loc[TRANSIT_BUS_ID]
    except KeyError: continue 
        
    temp_emissions = pd.DataFrame({'LinkID': df_activos['LinkID']})
    for pol in cols_presentes:
        temp_emissions[pol] = df_activos['Vol_Hora'] * df_activos['L_km'] * df_activos['SpeedBin'].map(factor_df[pol]).fillna(0)
        
    for h in horas:
        df_h = temp_emissions.copy()
        df_h['Hour'] = h + 1; df_h['Source'] = TRANSIT_BUS_ID; df_h['Day'] = 5 
        all_hourly_emissions.append(df_h)

df_final_dd = pd.concat(all_hourly_emissions, ignore_index=True)
df_final_dd = df_final_dd.groupby(['LinkID', 'Day', 'Hour', 'Source'], as_index=False)[cols_presentes].sum()
df_final_dd.to_parquet(INTEGRATION_DIR / "Transit_Emissions_DistBased_Day_5.parquet", index=False)
pd.DataFrame(geometry_records).to_parquet(INTEGRATION_DIR / "Transit_Geometry_Master.parquet", index=False)
print("   ✅ Emisiones DD y Geometría de TP guardadas.")

del all_hourly_emissions, df_final_dd, geometry_records; gc.collect()

# ==============================================================================
# 3. PROCESAMIENTO DI (DISTANCE INDEPENDENT)
# ==============================================================================
print("\n--- B. Calculando Emisiones Independientes de la Distancia (DI) TP ---")
network_data = []

for periodo, folder in FOLDER_MAPPING.items():
    file_prof = list((DIR_PROFILES / folder).glob("*.parquet"))
    file_lr = list((DIR_LR_ITEMS / folder).glob("*.parquet"))
    if not (file_prof and file_lr): continue
        
    df_prof, df_lr = pd.read_parquet(file_prof[0]), pd.read_parquet(file_lr[0])
    df_prof.columns, df_lr.columns = [c.strip().upper() for c in df_prof.columns], [c.strip().upper() for c in df_lr.columns]
    
    df_merged = pd.merge(df_lr, df_prof, on=['LINENAME', 'LINEROUTENAME', 'DIRECTIONCODE'], how='inner')
    df_merged = df_merged[df_merged['WKTPOLYWGS84'].notna() & (df_merged['WKTPOLYWGS84'] != 'LINESTRING EMPTY')]
    
    horas, hw_col = HOURS_MAPPING.get(periodo, []), f"HEADWAY{periodo}"
    if len(horas) == 0 or hw_col not in df_merged.columns: continue
        
    df_merged['HEADWAY_SEC'] = df_merged[hw_col].apply(clean_visum_string) * (1.0 if HEADWAY_IS_SECONDS else 60.0)
    df_merged['Vol_Hora'] = np.where(df_merged['HEADWAY_SEC'] > 0, 3600 / df_merged['HEADWAY_SEC'], 0)
    
    for _, row in df_merged[df_merged['Vol_Hora'] > 0].iterrows():
        link_id = wkt_to_id.get(row['WKTPOLYWGS84']) 
        if link_id:
            for h in horas: network_data.append((link_id, h + 1, TRANSIT_BUS_ID, row['Vol_Hora']))

df_net = pd.DataFrame(network_data, columns=['LinkID', 'Hour', 'Source', 'Link_Volume'])
df_net = df_net.groupby(['LinkID', 'Hour', 'Source'], as_index=False)['Link_Volume'].sum() 
df_net['Network_Fraction'] = df_net['Link_Volume'] / df_net.groupby(['Hour', 'Source'])['Link_Volume'].transform('sum')

df_rates = pd.read_parquet(RUTA_FACTORES_DI)
df_day = df_rates[df_rates['Day'] == 5].copy()
rate_cols = [c for c in df_rates.columns if c.startswith('Rate_')]

macro_cols = ['Hour', 'Source']
for col in rate_cols:
    base = col.replace('Rate_', '').replace('_per_veh', '')
    df_day[base] = df_day[col] * df_day['sourceTypePopulation']
    macro_cols.append(base)
    
df_final_di = pd.merge(df_net, df_day[macro_cols], on=['Hour', 'Source'], how='inner')
emission_cols = [c for c in macro_cols if c not in ['Hour', 'Source']]

for col in emission_cols: df_final_di[col] = df_final_di[col] * df_final_di['Network_Fraction']
df_final_di['Day'] = 5
df_final_di[['LinkID', 'Day', 'Hour', 'Source'] + emission_cols].to_parquet(INTEGRATION_DIR / "Transit_Emissions_VolBased_Day_5.parquet", index=False)
print("   ✅ Emisiones DI de TP guardadas.\n")

🚌 INICIANDO PROCESAMIENTO: TRANSPORTE PÚBLICO...

--- A. Calculando Emisiones Dependientes de la Distancia (DD) TP ---
   ✅ Emisiones DD y Geometría de TP guardadas.

--- B. Calculando Emisiones Independientes de la Distancia (DI) TP ---
   ✅ Emisiones DI de TP guardadas.



## 3. Unificación Total

In [4]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import os
import gc 
from pathlib import Path

# ==============================================================================
# 1. CONFIGURACIÓN DE UNIFICACIÓN
# ==============================================================================
BASE_DIR = Path(r"D:\NB3_VISUM plus MOVES merge\Integration Files V1")
GEOJSON_DIR = BASE_DIR / "Output Files" / "GeoJSON_Finales_Maestros"
GEOJSON_DIR.mkdir(parents=True, exist_ok=True)

# POLLUTANTS = ['CO2', 'CO2_Equiv', 'CO', 'NOx', 'Total_PM25', 'Total_PM10', 'TotalHC', 'CH4', 'N2O']
POLLUTANTS = ['CO2_Equiv']

def optimizar_memoria(df):
    df['LinkID'] = df['LinkID'].astype('int64') 
    df['Hour'] = pd.to_numeric(df['Hour'], downcast='integer')
    for col in POLLUTANTS:
        if col in df.columns: df[col] = pd.to_numeric(df[col], downcast='float')
    return df

# ==============================================================================
# 2. CARGA Y COMBINACIÓN ESPACIAL (WKT JOIN)
# ==============================================================================
print("🌍 INICIANDO LA GRAN UNIFICACIÓN (Auto + Carga + TP)...")

# --- A. Procesar Auto ---
print("   -> Integrando Auto y Carga...")
df_geom_auto = pd.read_parquet(BASE_DIR / "Links_Geometry_Master.parquet")
# 🌟 CORRECCIÓN AQUÍ: Forzar LinkID a entero en la geometría
df_geom_auto['LinkID'] = df_geom_auto['LinkID'].astype('int64')

df_dd_auto = optimizar_memoria(pd.read_parquet(BASE_DIR / "Link_Emissions_DistBased_Day_5.parquet"))
df_di_auto = optimizar_memoria(pd.read_parquet(BASE_DIR / "Link_Emissions_VolBased_Day_5.parquet"))

df_auto = pd.concat([df_dd_auto, df_di_auto], ignore_index=True)
df_auto = df_auto.groupby(['LinkID', 'Hour'], as_index=False)[POLLUTANTS].sum()
# Le pegamos el WKTPOLYWGS84 para poder sumar con los buses
df_auto = pd.merge(df_auto, df_geom_auto[['LinkID', 'WKTPOLYWGS84']], on='LinkID', how='inner')
df_auto.drop(columns=['LinkID'], inplace=True)
del df_geom_auto, df_dd_auto, df_di_auto; gc.collect()

# --- B. Procesar TP ---
print("   -> Integrando Transporte Público...")
df_geom_tp = pd.read_parquet(BASE_DIR / "Transit_Geometry_Master.parquet")
# 🌟 CORRECCIÓN AQUÍ: Forzar LinkID a entero en la geometría
df_geom_tp['LinkID'] = df_geom_tp['LinkID'].astype('int64')

df_dd_tp = optimizar_memoria(pd.read_parquet(BASE_DIR / "Transit_Emissions_DistBased_Day_5.parquet"))
df_di_tp = optimizar_memoria(pd.read_parquet(BASE_DIR / "Transit_Emissions_VolBased_Day_5.parquet"))

df_tp = pd.concat([df_dd_tp, df_di_tp], ignore_index=True)
df_tp = df_tp.groupby(['LinkID', 'Hour'], as_index=False)[POLLUTANTS].sum()
df_tp = pd.merge(df_tp, df_geom_tp[['LinkID', 'WKTPOLYWGS84']], on='LinkID', how='inner')
df_tp.drop(columns=['LinkID'], inplace=True)
del df_geom_tp, df_dd_tp, df_di_tp; gc.collect()

# --- C. Fusión Total y Agrupación Espacial ---
print("   -> Fusionando Flotas sobre la misma geometría física...")
df_master = pd.concat([df_auto, df_tp], ignore_index=True)
# Agrupamos por WKT y Hora. Si un bus y un auto pasan por la misma calle, se suman.
df_master = df_master.groupby(['WKTPOLYWGS84', 'Hour'], as_index=False)[POLLUTANTS].sum()
del df_auto, df_tp; gc.collect()

# ==============================================================================
# 3. CREACIÓN DE GEOMETRÍA Y CÁLCULO PERFECTO DE LONGITUD
# ==============================================================================
print("   -> Transformando a GeoDataFrame...")
df_master['geometry'] = df_master['WKTPOLYWGS84'].astype(str).apply(wkt.loads)
df_master.drop(columns=['WKTPOLYWGS84'], inplace=True)

gdf_final = gpd.GeoDataFrame(df_master, geometry='geometry')
gdf_final.set_crs(epsg=4326, inplace=True)

print("   -> Recalculando longitud física exacta (UTM Monterrey)...")
# El cálculo de longitud real en UTM 14N (EPSG:32614) que arregla la visualización de densidad
gdf_final['L_km_real'] = gdf_final.to_crs(epsg=32614).geometry.length / 1000.0
# Imponemos un mínimo de 1 metro para evitar divisiones por cero o atípicos (incluso si Visum falló)
gdf_final['L_km_real'] = gdf_final['L_km_real'].clip(lower=0.001)

print("   -> Calculando densidad (g/km)...")
for pol in POLLUTANTS:
    gdf_final[f'Densidad_{pol}_g_km'] = gdf_final[pol] / gdf_final['L_km_real']
    gdf_final[f'Densidad_{pol}_g_km'] = gdf_final[f'Densidad_{pol}_g_km'].fillna(0).replace([np.inf, -np.inf], 0)

# ==============================================================================
# 4. EXPORTACIÓN
# ==============================================================================
print("\n🗺️ Generando GeoJSONs Maestros por Contaminante...")
for pol in POLLUTANTS:
    gdf_export = gdf_final[gdf_final[pol] > 0].copy()
    
    if not gdf_export.empty:
        export_cols = ['Hour', 'L_km_real', pol, f'Densidad_{pol}_g_km', 'geometry']
        filename = GEOJSON_DIR / f"Mapa_TOTAL_Flota_{pol}.geojson"
        print(f"   💾 Guardando {filename.name} (Calles activas: {len(gdf_export):,})...")
        gdf_export[export_cols].rename(columns={pol: f'Total_{pol}_g'}).to_file(filename, driver='GeoJSON')

print(f"\n🚀 ¡ÉXITO ROTUNDO! Proceso completado. Revisa tu carpeta: \n{GEOJSON_DIR}")

🌍 INICIANDO LA GRAN UNIFICACIÓN (Auto + Carga + TP)...
   -> Integrando Auto y Carga...
   -> Integrando Transporte Público...
   -> Fusionando Flotas sobre la misma geometría física...
   -> Transformando a GeoDataFrame...
   -> Recalculando longitud física exacta (UTM Monterrey)...
   -> Calculando densidad (g/km)...

🗺️ Generando GeoJSONs Maestros por Contaminante...
   💾 Guardando Mapa_TOTAL_Flota_CO2_Equiv.geojson (Calles activas: 1,533,044)...

🚀 ¡ÉXITO ROTUNDO! Proceso completado. Revisa tu carpeta: 
D:\NB3_VISUM plus MOVES merge\Integration Files V1\Output Files\GeoJSON_Finales_Maestros
